# Итоговое задание по Проекту 3. О вкусной и здоровой пище

### В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor 
### Задание выполнил Бегунов Павел (DST-56)

# Импорт библиотек

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from datetime import datetime, timedelta
import re

from sklearn.model_selection import train_test_split  # загружаем специальный удобный инструмент для разделения датасета
from sklearn.preprocessing import LabelEncoder # инструмент для кодирования категориальных признаков
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# Предобработка

In [ ]:
def rename_columns(df):  
    ''' 
    The function converts the column names to a single format. 
    '''
    
    columns = {}
    for column in df.columns:
        columns[column] = column.lower().replace(' ', '_')
    return df.rename(columns=columns, inplace=True)


def missing_data(df):
    '''
    The function displays the number and percentage of skips for each column.
    '''
    for column in df.columns:
        missing = df[column].isnull().sum()
        percent_of_missing = np.mean(df[column].isnull())
        if missing > 0:
            print('{} - {} value(s), {:.2f}%'.format(column, missing, percent_of_missing*100))
            
            
def IQR_outlier(column, verbose=True):
    '''
    The function displays the boundaries of the interquartile range.
    '''
    perc25 = df[column].quantile(0.25)
    perc75 = df[column].quantile(0.75)
    IQR = perc75 - perc25
    low = perc25 - 1.5*IQR
    high = perc75 + 1.5*IQR
    anomaly = len(df[df[column] > high]) + \
        len(df[df[column] < low])
    if verbose:
        print('25-й перцентиль: {},'.format(perc25)[:-1], '75-й перцентиль: {},'.format(perc75),
            'IQR: {}, '.format(IQR), 'Границы выбросов: [{f}, {l}].'.format(f=low, l=high))
        print('Выбросов, согласно IQR: {} | {:2.2%}'.format(
            anomaly, anomaly/len(df)))

# Импорт данных

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

rename_columns(df_train)
rename_columns(df_test)
rename_columns(sample_submission)

display(df_train.head(5))
display(df_test.head(5))
display(sample_submission.head(5))

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.sample(5)

# Обзор данных

## Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:
* restaurant_id: идентификационный номер ресторана / сети ресторанов;
* city: Город; 
* cuisine_style: Кухня;
* ranking: Ранг ресторана относительно других ресторанов в этом городе;
* price_range: Цены в ресторане в 3 категориях;
* number_of_reviews: Количество отзывов;
* reviews: 2 последних отзыва и даты этих отзывов;
* url_ta: страница ресторана на 'www.tripadvisor.com'; 
* id_ta: ID ресторана в TripAdvisor;
* rating: Рейтинг ресторана;

In [ ]:
df_train.info()

In [ ]:
print("Размер датасета для обучения: {}".format(df_train.shape))

In [ ]:
df_test.info()

In [ ]:
print("Размер датасета для тестирования: {}".format(df_test.shape))

In [ ]:
df.info()

In [ ]:
print("Размер оьбъединенного датасета: {}".format(df.shape))

In [ ]:
# Проверим данные на наличие пропусков:
missing_data(df_train)

In [ ]:
#Проверим данные на наличие дубликатов:
if len(df_train)>len(df_train.drop_duplicates()):
    print('Duplicates found')
    display(df_train[df_train.duplicated()])
else:
    print('Duplicates not found')

In [ ]:
# посмотрим какие признаки у нас могут быть категориальными:
df_train.nunique(dropna=False)

**Резюме**: Имеется два датасета (df_train и df_test). Первичный обзор данных показал, что для объектов первого известна целевая переменная rating (рейтинг ресторанов). Определены признаки, которые могут быть категориальными. Большинство признаков требуют очистки и предварительной обработки.  На основании обучающего датасета (df_train) необходимо создать модель и предсказать значение рейтинга ресторана для объектов из тестового датасета (df_test).  В качестве оценки качества модели используется MAE (средняя абсолютная ошибка), таким образом в результате работы необходимо получить наименьшее значение MAE.

# Детальный анализ по переменным

### 1. restaraunt_id

In [ ]:
df_train['restaurant_id'].apply(lambda x: x[3:]).astype('int64').hist(figsize=(10,5), bins=100, color='g');

**Резюме:** Следует проверить насколько данный признак информативен.

### 2. city

In [ ]:
# посмотрим на количество уникальных значений:
df_train['city'].nunique()

In [ ]:
# определим количество ресторанов в каждом городе
df['restaurants_count'] = df['city'].map(df.groupby('city')['restaurant_id'].count().to_dict())

In [ ]:
# посмотрим распределение ресторанов по городам:
plt.subplots(figsize=(12, 7))
plt.xticks(ticks=range(0, 7501, 500))
plt.title('Распределение ресторанов по городам')
sns.countplot(y="city",
              data=df_train,
              order=df_train['city'].value_counts().index)
plt.xlabel('Количество ресторанов')
plt.ylabel('Город');

In [ ]:
сity_dummies = pd.get_dummies(df['city'], dummy_na=False).astype('float64')
df = pd.concat([df,сity_dummies], axis=1)

In [ ]:
le = LabelEncoder()
le.fit(df['city'])
df['code_сity'] = le.transform(df['city'])

### 2.1 city_population

In [ ]:
# создадим словарь с городами и населением в них (в млн. человек)
cities = list(sorted(df['city'].unique()))
population = [0.825080, 3.168846, 0.424819, 3.326002, 0.413192, 0.176545, 1.744665, 1.246611, 0.506211,
              0.482005, 0.198899, 1.718187, 0.574579, 0.756183, 0.547733, 0.277554, 8.538689, 0.214,
              0.506615, 3.155360, 1.331586, 1.561094, 0.258975, 0.673469, 2.148327, 1.272690,  2.870493,
              0.961609, 1.765649, 1.720398, 0.402275]
city_population_dict = dict(zip(cities, population))
df['city_population'] = df['city'].map(lambda x: city_population_dict[x])
# создадим признак количество ресторанов на 10_000 человек
df['restaurants_for_population'] = df['restaurants_count'] / (df['city_population'] * 10_000)

### 2.2 city_area

In [ ]:
# создадим словарь с городами и их площадью (в кв. км)
area = [219.4, 412, 101.3, 891.68, 368, 32.61, 525.14, 86.40, 318, 118, 15.93, 755.09, 213.8, 327, 100.05,
        163.8, 1572, 51.47, 47.87, 607, 181.67, 310.71, 41.66, 454, 105.4, 496, 1287.36, 188, 414.75, 517, 91.88]
city_area_dict = dict(zip(cities, area))
df['city_area'] = df['city'].map(lambda x: city_area_dict[x])
# создадим признак плотность ресторанов на кв.км
df['restaurant_density'] = df['restaurants_count'] / df['city_area']

**Резюме:** Категориальный признак. 31 уникальное значение, пропусков нет . Сгенерированы новые признаки города по типу dummies, также создан новый критерий code_сity с кодировкой LabelEncoder из библиотеки sklearn.

Добавлены новые критерии:
* code_city - код города с помощью LabelEncoder
* city_area - площадь города (кв.км)
* сity_population - население города (млн.чел.)
* restaurants_for_population - кол-во ресторанов на 10_000 человек
* restaurant_density - кол-во ресторанов на 1 кв.км

### 3. cuisine_style

In [ ]:
# в данном признаке  9283 (23.21%) пропущенных значений, сохраним данную информацию
df['NaN_cuisine_style'] = pd.isna(df['cuisine_style']).astype('int8')
# заменим пропуски значением 'Traditional', предполагая, что кухня соответствует традиционной для данной местности
df['cuisine_style'] = df['cuisine_style'].fillna('Traditional')
# очистим данные от посторонних знаков и сформируем списки кухонь, разбив данные по запятым
df['cuisine_style'] = df['cuisine_style'].apply(lambda x: str(x).replace(
    "[", "").replace("]", "").replace("'", "").replace(" ", "").split(","))
# сформируем новый признак: кол-во кухонь в ресторане
df['cuisine_variety'] = df['cuisine_style'].map(lambda x: len(x))

In [ ]:
# посмотрим на распределение признака
plt.figure(figsize=(12, 6))
plt.xticks(ticks=range(1, 25, 1))
plt.title('Распределение количества представленных кухонь')
sns.countplot(df['cuisine_variety'], color='g', saturation=0.5)
plt.xlabel('Количество кухонь')
plt.ylabel('Число ресторанов');

In [ ]:
# проверим признак на наличие выбросов
IQR_outlier('cuisine_variety')

In [ ]:
# посмотрим на распределение признака, значения которого выходят за границы IQR анализа
plt.figure(figsize=(12, 6))
plt.xticks(ticks=range(1, 25, 1))
plt.title('Распределение количества представленных кухонь > 8')
plt.xlabel('Количество кухонь')
plt.ylabel('Чисто ресторанов')
sns.countplot(df[df['cuisine_variety'] > 8]['cuisine_variety'], color='g', saturation=0.5);

**Резюме:** Большое количество пропусков 9283 (23.21%). Чтобы сохранить эту информацию сформирован новый признак 'NaN_cuisine_style'. После обработки, добавлен критерий количества типов кухонь в ресторане 'cuisine_variety'. Несмотря на то, что  IQR анализ показал наличие выбросов в новом признаке в количестве 126. При детальном рассмотрении принято решение не относить эти значения к выбросам.

### 4. ranking

In [ ]:
# посмотрим распредение ресторанов по рангу
df_train['ranking'].hist(figsize=(10,5), bins=100, color='g');

In [ ]:
# посмотрим на признак ranking топ 10 городов
for x in (df_train['city'].value_counts())[0:10].index:
    df_train['ranking'][df_train['city'] == x].hist(figsize=(10,6), bins=100)

In [ ]:
ranking_on_city = df.groupby('city')['ranking'].agg(['max', 'min'])

In [ ]:
# сгенерируем новый признак с нормализованным по городам рангом ресторанов
df['norm_ranking_on_city'] = df[['city', 'ranking']].apply(lambda x: (
    x[1] - ranking_on_city['min'][x[0]]) / (ranking_on_city['max'][x[0]] - ranking_on_city['min'][x[0]]), axis=1)

In [ ]:
# данные по нормализованному Ranking топ 10-ти городов
for x in (df['city'].value_counts())[0:10].index:
    df['norm_ranking_on_city'][df['city'] == x].hist(figsize=(10,6), bins=100)

**Резюме:** Распределение ресторанов по рангу очень схоже с распределением признака restaurant_id. Нужно посмотреть корреляцию данных признаков и в случае необходимости удалить наименее информативный. Получилось, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за этого имело место смещение. Для того, чтобы при обучении модели не возникало ошибок была проведена нормализация признака по городам по методу минимакс. 

Сгенерирован новый признак:
* norm_ranking_on_city - нормализованный по городам ранг ресторанов. 

### 5. price_range

In [ ]:
# признак price_range имеет множество пропущенных значений 13886 (34.72%), сохраним данную информацию
df['NaN_price_range'] = pd.isna(df['price_range']).astype('int8')
# заменим значения на числовые:
dict_value_price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['price_range'] = df['price_range'].map(dict_value_price)

In [ ]:
display(df['NaN_price_range'])
df['NaN_price_range'].value_counts()

In [ ]:
# посмотрим на распределения признака price_range:
sns.countplot(data=df, x='price_range');

In [ ]:
# заменим пропуски на максимально часто встречающийся элемент данного признака
df['price_range'] = df['price_range'].fillna(2)

In [ ]:
# price_range_dummies = pd.get_dummies(df['price_range'], dummy_na=False).astype('float64')
# df = pd.concat([df,price_range_dummies], axis=1)

**Резюме:** в переменной пропущено 13886 значений (34.72%). Чтобы сохранить эту информацию сформирован новый признак 'NaN_price_range'. Также создан новый признак числового кодирования цены - 'price_range' (низкая цена - 1, средняя цена -2 , высокая цена - 3). Пропуски заполнили модой.

### 6. number_of_reviews

In [ ]:
# признак number_of_reviews имеет  2543 пропущенных значений (6.36%), сохраним данную информацию
df['NaN_number_of_reviews'] = pd.isna(df['number_of_reviews']).astype('int8')

In [ ]:
df['NaN_number_of_reviews'].value_counts()

In [ ]:
# заполняем пропуски несколькими вариантами: 
# значением равным 0 (по условию)
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)

# средним значением признака number_of_reviews
#  df['number_of_reviews'] = df['number_of_reviews'].fillna(df['number_of_reviews'].mean())

# средним значением по городу
# mean_number_of_reviews_on_city = df.groupby('city')['number_of_reviews'].mean()
# df['number_of_reviews'] = df[['city', 'number_of_reviews']].apply(
#     lambda x: mean_number_of_reviews_on_city[x[0]] if pd.isna(x[1]) else x[1], axis=1)

In [ ]:
# посмотрим на распределение по количеству отзывов
df['number_of_reviews'].hist(figsize=(10,5), bins=50)
plt.ylabel('Количество ресторанов');

In [ ]:
# проверим признак на наличие выбросов
IQR_outlier('number_of_reviews')

In [ ]:
# согласно IQR 11.89% данных являются выбросами
# логарифмируем, чтобы сгладить распределение и тем самым уменьшить количество аномальных значений
df['log_number_of_reviews'] = np.log(df['number_of_reviews'] + 1)
IQR_outlier('log_number_of_reviews')

In [ ]:
# посмотрим на распределение
df['log_number_of_reviews'].hist(figsize=(10,5), bins=50)
plt.ylabel('Количество ресторанов');

In [ ]:
# создадим новый признак frequency_of_reviews. количество отзывов на 100_000 человек
df['frequency_of_reviews'] = df['log_number_of_reviews'] / df['city_population'] *10

**Резюме:**  в данном признаке 2543 пропущенных значений (6.36%). Пропуски были заполнены нулями, так как данная замена показала себя наилучшим образом как при IQR анализе, так и при определении метрики. Прологорифмированное распределение имеет не стандартный вид, хотя заметно улучшилось. Согласно IQR осталось 12 выбросов, что составляет 0,02%.

### 7. reviews

In [ ]:
# посмотрим на данные
df['reviews'].value_counts(dropna=False)

In [ ]:
# заполним пропуски самым популярным значением
df['reviews'].fillna('[[], []]', inplace = True)

In [ ]:
# cоздаем столбец review_dates содержащий список дат отзывов
df['review_dates'] = df['reviews'].str.findall('\d+\/\d+\/\d+')
df['review_dates'].sample(5)

In [ ]:
# посмотрим на данные по датам
df['number_of_date'] = df['review_dates'].apply(lambda x: len(list(x)))
df['number_of_date'].value_counts()

In [ ]:
# посмотрим на данные с 3 датами
df[df['number_of_date'] == 3]['reviews']

In [ ]:
# удалим даты из отзывов
df['review_dates'] = df['review_dates'].apply(lambda x: x if len(x) <= 2 else x[1:])

In [ ]:
# разделим признак review_dates на first_review (дата первого отзыва) и last_review (дата последнего отзыва)
df['first_review'] = df['review_dates'].apply(lambda x: pd.to_datetime(x).min())
df['last_review'] = df['review_dates'].apply(lambda x: pd.to_datetime(x).max())
# cоздадим признак difference_dates_of_reviews с разницей между первым и вторым отзывом в днях
df['difference_dates_of_reviews'] = (df['last_review'] - df['first_review']).apply(lambda x: x.days)
# Заменим пропуски средним значением и округлим до целого
df['difference_dates_of_reviews'].fillna(round(df['difference_dates_of_reviews'].mean()), inplace=True)

In [ ]:
IQR_outlier('difference_dates_of_reviews')

In [ ]:
# удалим first_review и last_review
df.drop(['first_review','last_review'], axis=1, inplace=True)

**Резюме:** Пропусков в df_train нет, в df_test - 2. Но есть 8112 незаполненных строк с отзывами (16,2%). В 5680 (14%) строках есть только один отзыв, хотя в подавляющем большинстве отзывов два. IQR анализ указывает на наличие 4803(9,61%) выбросов.

Созданы новые критерии:


* number_of_date - кол-во дат в отзыве
* difference_dates_of_reviews - разница между отзывами в днях

### 8. url_ta

In [ ]:
# посмотрим на данные
df['url_ta'][0]

In [ ]:
# удалим данный признак
df.drop('url_ta', axis=1, inplace=True)

**Резюме:** в колонке указаны ссылки ресторанов на сайте TripAdvisor. Данный признак полезен для получения новой информации о ресторанах, генерации новых признаков, и, возможно, для запонения пропусков в данных. Однако мне не хватает времени для парсинга, поэтому данный признак был исключен из дальнейшего рассмотрения как не информативный.

### 9. id_ta 

In [ ]:
# посмотрим на данные
df['id_ta'][0]

In [ ]:
# удалим данный признак
df.drop('id_ta', axis=1, inplace=True)

**Резюме:** вероятно это технические данные для верификации ссылок на рестораны, так как значения id_ta присутствуют в ссылках из url_ta. Для построения модели признак не информативен.

### 10. rating

In [ ]:
# посмотрим распределение целевой переменной
df_train['rating'].value_counts(ascending=True).plot(figsize=(10, 5), kind='barh');

In [ ]:
# посмотрим распределение целевой переменной относительно признака
df_train['ranking'][df_train['rating'] == 5].hist(figsize=(10, 5), bins=100);

In [ ]:
df_train['ranking'][df_train['rating'] < 4].hist(figsize=(10, 5), bins=100);

# Корреляция числовых переменных

In [ ]:
# Сформируем список признаков, которые исключаем из корреляционного анализа
cols_to_drop = ['sample', 'Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava', 'Brussels',
                'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh', 'Geneva', 'Hamburg', 'Helsinki',
                'Krakow', 'Lisbon', 'Ljubljana', 'London', 'Luxembourg', 'Lyon', 'Madrid','Milan',
                'Munich', 'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm', 'Vienna', 'Warsaw', 'Zurich',
                'NaN_cuisine_style', 'NaN_price_range', 'NaN_number_of_reviews']

In [ ]:
plt.rcParams['figure.figsize'] = (15,15)
df_corr = df.copy()
sns.heatmap(df_corr[df_corr['sample']== 1].drop(cols_to_drop, axis=1).corr(), square=True,
            annot=True, fmt=".2f", linewidths=0.1, cmap="RdBu");

**Резюме:** С целевой переменной больше всего коррелируют ranking и norm_ranking_on_city. Показатель city_area имеет сильную корреляцию с признаком city_population, поэтому его можно убрать.

# Удаление признаков

In [ ]:
# Убираем коллинераные признаки:
df.drop('city_area', axis = 1, inplace=True)

# убираем не числовые признаки 
object_columns = [s for s in df.columns if df[s].dtypes == 'object']
df.drop(object_columns, axis = 1, inplace=True)

In [ ]:
df.head(4)

# Разбиваем датасет на тренировочный и тестовый

In [ ]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data['rating'].values  # наш таргет
X = train_data.drop(['rating'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Обучаем модель, генерируем результат и сравниваем с тестом

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh');

# Submission

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)
# Так как признак рейтинга имеет шаг 0.5, округляем предсказание.
predict_submission = np.round(predict_submission * 2) / 2
predict_submission

In [ ]:
sample_submission['rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)